<h1>Prepare Data(Calculate all timeframe from 1m timeframe data)!</h1>
<p>To calculate all available timeframes (2min, 3min, 5min, 15min, etc.) on Binance using 1-minute data, a slicing window method can be applied. By aggregating the 1-minute data into desired timeframes, you can generate higher intervals, allowing for flexible analysis of price trends and market patterns over varying timeframes.</p>

In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
from numpy.lib.stride_tricks import sliding_window_view

<h3>The 'prepare_data' class!</h3>
<ol>
    <li>
        <h3>The '__init__' function:</h3>
        Initialize a few important parameters.
        <ul>
            <li>'self.paths': Contains a list of all trading pairs for which data has been downloaded.</li>
            <li>'self.timeframes' : A dict that save the window size correspond to time frame when compared to the 1 min timeframe.</li>
        </ul>
    </li>
    <li>
        <h3>The 'handle_ohlcv' function:</h3>
        Caculate open,high,low,close,timestamp,volume,date of the candle.
        <ul>
            <li>The 'arr[0,0]','arr[0,1]','arr[0,2]','arr[0,3]' : Get the 'Date','Timestamp','Timeframe','Open' value.</li>
            <li>'np.max(arr[:,4])' : for the max value on each candle ('High').</li>
            <li>'np.min(arr[:,5])' : for the min value off the candle ('Low').</li>
            <li>'arr[-1:6]' : The close value of the candle ('Close').</li>
            <li>'np.sum(arr[:,7])' : Total trading volume in that timeframe.
        </ul>
    </li>
    <li>
        <h3>The 'get_timeframe' function:</h3>
        Calculate the different timeframes of each pair.
        <ul>
            <li>For each trading pair,for each timeframe of that pair.</li>
            <li>'wsize' variable : get the window size(the timeframe size compare with the 1min timeframe).</li>
            <li>The 'sliding_window_view' function use the window with size ('wsize',8) to slide from top to bottom of the 'df' variable.After that,we'll jump a distance equal to 'wsize' (eg :3min timeframe will have consecutive time values 20:00:00 ->20:03:00 and not 20:00:00 ->20:01:00).</li>
            <li>After sliding and jumping,we set index,timeframe and then save with csv format.</li>
        </ul>
    </li>
</ol>

In [2]:
class prepare_date():
    def __init__(self):        
        self.paths=os.listdir(os.path.join(Path().cwd(),'data'))
        self.timeframes={'1m':1,'3m':3,'5m':5,'15m':15,'30m':30,'1h':60,'2h':120,'4h':240,'6h':360,'8h':480,'12h':720,'1d':1440}    
    def handle_ohlcv(self,arr):
        return [arr[0,0],arr[0,1],arr[0,2],arr[0,3],np.max(arr[:,4]),np.min(arr[:,5]),arr[-1,6],np.sum(arr[:,7])]
    def get_timeframe(self):        
        for pair in self.paths:
            for tf in self.timeframes.keys():
                if not os.path.exists(os.path.join(Path().cwd(),f"data/{pair}/{pair}_{tf}.csv")):
                    print(pair,tf)
                    df=pd.read_csv(os.path.join(Path().cwd(),f"data/{pair}/{pair}_1m.csv"))
                    wsize=self.timeframes[tf]
                    slide=sliding_window_view(df.values,(wsize,8)).squeeze()[::wsize]
                    get_timeframe=np.array(list(map(self.handle_ohlcv,slide)))
                    df=pd.DataFrame(get_timeframe,columns=['Date','Timestamp','Timeframe','Open','High','Low','Close','Volume'])
                    df.set_index('Date',inplace=True)
                    df.loc[:,'Timeframe']=tf
                    df.to_csv(os.path.join(Path().cwd(),f"data/{pair}/{pair}_{tf}.csv"))


In [3]:
prepare=prepare_date()
prepare.get_timeframe()

BNBUSDT 3m
BNBUSDT 5m
BNBUSDT 15m
BNBUSDT 30m
BNBUSDT 1h
BNBUSDT 2h
BNBUSDT 4h
BNBUSDT 6h
BNBUSDT 8h
BNBUSDT 12h
BNBUSDT 1d
ETHUSDT 3m
ETHUSDT 5m
ETHUSDT 15m
ETHUSDT 30m
ETHUSDT 1h
ETHUSDT 2h
ETHUSDT 4h
ETHUSDT 6h
ETHUSDT 8h
ETHUSDT 12h
ETHUSDT 1d
BTCUSDT 3m
BTCUSDT 5m
BTCUSDT 15m
BTCUSDT 30m
BTCUSDT 1h
BTCUSDT 2h
BTCUSDT 4h
BTCUSDT 6h
BTCUSDT 8h
BTCUSDT 12h
BTCUSDT 1d
